# Biostat 257 Homework 2

**Due May 1 @ 11:59PM**

In [1]:
versioninfo()

Julia Version 1.4.1
Commit 381693d3df* (2020-04-14 17:20 UTC)
Platform Info:
  OS: Windows (x86_64-w64-mingw32)
  CPU: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-8.0.1 (ORCJIT, skylake)
Environment:
  JULIA = D:\Julia-1.4.0\bin


Consider a linear mixed effects model
$$
    \mathbf{Y}_i = \mathbf{X}_i \boldsymbol{\beta} + \mathbf{Z}_i \boldsymbol{\gamma} + \boldsymbol{\epsilon}_i, \quad i=1,\ldots,n,
$$
where   
- $\mathbf{Y}_i \in \mathbb{R}^{n_i}$ is the response vector of $i$-th individual,  
- $\mathbf{X}_i \in \mathbb{R}^{n_i \times p}$ is the fixed effect predictor matrix of $i$-th individual,  
- $\mathbf{Z}_i \in \mathbb{R}^{n_i \times q}$ is the random effect predictor matrix of $i$-th individual,  
- $\boldsymbol{\epsilon}_i \in \mathbb{R}^{n_i}$ are multivariate normal $N(\mathbf{0}_{n_i},\sigma^2 \mathbf{I}_{n_i})$,  
- $\boldsymbol{\beta} \in \mathbb{R}^p$ are fixed effects, and  
- $\boldsymbol{\gamma} \in \mathbb{R}^q$ are random effects assumed to be $N(\mathbf{0}_q, \boldsymbol{\Sigma}_{q \times q}$) independent of $\boldsymbol{\epsilon}_i$.

## Q1 Formula (10 pts)

Write down the log-likelihood of the $i$-th datum $(\mathbf{y}_i, \mathbf{X}_i, \mathbf{Z}_i)$ given parameters $(\boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma^2)$.

**Solution:** 

From givin information we know, $\bar {Y} = X\beta$ and $Var(Y) = \sigma^2I_n + Z\Sigma Z^T$, and thus we can calculate the log likelihood function as:
\begin{equation}\begin{split}
l & = -\frac{n}{2}\ln(2\pi)-\frac{1}{2}\ln( |Var(Y_i)|) - \frac{1}{2}(Y-\mu)^TVar(Y_i)^{-1}(Y-\mu)\\
& = -\frac{n}{2}\ln(2\pi)-\frac{1}{2}\ln (|\sigma^2I_n + Z\Sigma Z^T|) - \frac{1}{2}(Y-X\beta)^T(\sigma^2I_n + Z\Sigma Z^T)^{-1}(Y-X\beta)
\end{split}\end{equation}
In this case we have the **Variance** of y as: 
$$\sigma^2I_n + Z\Sigma Z^T = \sigma^2I_n + ZLL^TZ^T$$
which can be recognized as an "easy + low rank" structure.


Also we can identify $A = \sigma^2I_n$ and $U = V = ZL$ and thus utilize the **Woodbury formula** and determinantal fomula in HW1 Q5. By Woodbury, the inverse of the covariance matrix is:
$$(\sigma^2I_n + ZLL^TZ^T)^{-1} = \sigma^{-2}I_n - \sigma^{-4}ZL(I_q + \sigma^{-2}L^TZ^TZL)^{-1}L^TZ^T$$
And we can calculate the $(ZL)^TZL$ in the order of $Z^TZ$ (q\*n X n\*q => q\*q), $L^TZ^TZ$ (q\*q X q\*q), $L^TZ^TZL$ (q\*q X q\*q) to get the lowest computational cost. Proper implementation of this function will cost $\mathcal{O}(np)$ (calculating the residual vector $r = y-X\beta$) plus $\mathcal{O}(nq)$ (calculating $Z^Tr$) flops.

And we can further cut the cost by pre-computing certain quantities in light of identities
$$r^Tr = y^Ty + \beta^TX^TX\beta - 2y^TX\beta$$
and
$$Z^Tr = Z^Ty - Z^TX\beta$$
So each evaluation costs $\mathcal{O}(p^2) + \mathcal{O}(q^3) +\mathcal{O}(pq)$ flops, and in real application, these variables are all small numbers.

**Note:** we can only pre-allocate/pre-calculate varibales related to data, like X, y, but those variable related with model parameters should not be pre-calculated, since we need to adjust these parameters to validate the model. 

## Q2 Start-up code

Use the following template to define a type `LmmObs` that holds an LMM datum $(\mathbf{y}_i, \mathbf{X}_i, \mathbf{Z}_i)$. 

In [2]:
# define a type that holds LMM datum
struct LmmObs{T <: AbstractFloat}
    # data
    y :: Vector{T}
    X :: Matrix{T}
    Z :: Matrix{T}
    # working arrays
    # whatever intermediate arrays you may want to pre-allocate
#     yty         :: T
#     xty         :: Vector{T}
#     zty         :: Vector{T}
#     xtx         :: Matrix{T}
#     ztx         :: Matrix{T}
    res         :: Vector{T}
    storage_q   :: Vector{T}
    storage_q2  :: Vector{T}
    ztz         :: Matrix{T}
    storage_qq  :: Matrix{T}
end

# constructor
function LmmObs(
        y::Vector{T}, 
        X::Matrix{T}, 
        Z::Matrix{T}) where T <: AbstractFloat
#     yty        = abs2(norm(y))
#     xty        = transpose(x) * y
#     zty        = transpose(z) * y
#     xtx        = transpose(x) * x
#     ztx        = transpose(x) * x
    res        = similar(y)
    storage_q  = Vector{T}(undef, size(Z, 2))
    storage_q2 = Vector{T}(undef, size(Z, 2))
    ztz        = transpose(Z) * Z
    storage_qq = similar(ztz)
    LmmObs(y, X, Z, res, storage_q, storage_q2, ztz, storage_qq)
#     LmmObs(y, X, Z, yty, xty, zty, xtx, ztx,
#         storage_q, storage_q2, ztz, storage_qq)
end

LmmObs

Write a function, with interface   
```julia
logl!(obs, β, L, σ²)
```
that evaluates the log-likelihood of the $i$-th datum. Here `L` is the lower triangular Cholesky factor from the Cholesky decomposition `Σ=LL'`. Make your code efficient in the $n_i \gg q$ case. Think the intensive longitudinal measurement setting.  

In [3]:
function logl!(
        obs :: LmmObs{T}, 
        β   :: Vector{T}, 
        L   :: Matrix{T}, 
        σ²  :: T) where T <: AbstractFloat
    n, p, q = size(obs.X, 1), size(obs.X, 2), size(obs.Z, 2)    

    # TODO: compute and return the log-likelihood
    
    # Calculate residual
    BLAS.gemv!('N', T(1), obs.X, β, T(-1), copy!(obs.res, obs.y))
    # IF the structure obs.res = copy(y), but this will make res a shadow of y, and y will change with res
#     BLAS.gemv!('N', T(1), obs.X, β, T(-1), obs.res)
    
#     mul!(obs.res, obs.X, β)  # bottleneck
#     obs.res .= obs.y .- obs.res
    
    #Z'A => L'Z'A  = (ZL)'A
    mul!(obs.storage_q2, transpose(obs.Z), obs.res)
    mul!(obs.storage_q, transpose(L), obs.storage_q2)
    
    # calculate (ZL)'ZL
    mul!(obs.storage_qq, transpose(L), obs.ztz)
    rmul!(obs.storage_qq,LowerTriangular(L))
    
    rdiv!(obs.storage_qq, σ²)
    # obs.storage_qq ./= σ² 
    for j = 1:q
        obs.storage_qq[j,j] += 1
    end
    # obs.storage_qq .+= Matrix{Float64}(I, q, q)
    Ωchol = cholesky!(Symmetric(obs.storage_qq))
    
#     # determinate of the triangular matrix, also det(LU) = det(L)^2
#     det = 1
#     for j = 1:q
#         det *= obs.storage_qq[j,j] 
#     end
    
    l= (-(n//2) * log(2π)
#         - (1//2)*( log(det^2)+n*log(σ²) )
        - (1//2)*( logdet(Ωchol) + n*log(σ²) )
        - (1//2)*( dot(obs.res, obs.res)/σ² -
            (1/(σ²*σ²))*dot(obs.storage_q, Ωchol\obs.storage_q) )
    )

    return l
end

logl! (generic function with 1 method)

**Hint**: This function shouldn't be very long. Mine, obeying 80-character rule, is 25 lines. If you find yourself writing very long code, you're on the wrong track. Think about algorithm first then use BLAS functions to reduce memory allocations.

## Q3 Correctness (15 pts)

Compare your result (both accuracy and timing) to the [Distributions.jl](https://juliastats.org/Distributions.jl/stable/multivariate/#Distributions.AbstractMvNormal) package using following data.

In [4]:
using BenchmarkTools, Distributions, LinearAlgebra, Random

Random.seed!(257)
# dimension
n, p, q = 2000, 5, 3;

# predictors
X  = [ones(n) randn(n, p - 1)];
Z  = [ones(n) randn(n, q - 1)];
# parameter values
β  = [2.0; -1.0; rand(p - 2)];
σ² = 1.5;
Σ  = fill(0.1, q, q) + 0.9I;
# generate y
y  = X * β + Z * rand(MvNormal(Σ)) + sqrt(σ²) * randn(n);

# form an LmmObs object
obs = LmmObs(y, X, Z);

This is the standard way to evaluate log-density of a multivariate normal, using the Distributions.jl package. Let's evaluate the log-likelihood of this datum.

In [5]:
μ  = X * β
Ω  = Z * Σ * transpose(Z) +  σ² * I

# From istribution pacakage:
mvn = MvNormal(μ, Symmetric(Ω)) # MVN(μ, Σ)
logpdf(mvn, y)

-3247.456858063827

Check that your answer matches that from Distributions.jl

In [6]:
L = Matrix(cholesky(Σ).L)
logl!(obs, β, L, σ²)

-3247.4568580638293

**You will lose all 15 + 30 + 30 = 75 points** if the following statement throws `AssertError`.

In [7]:
@assert logl!(obs, β, Matrix(cholesky(Σ).L), σ²) ≈ logpdf(mvn, y)

## Q4 Efficiency (30 pts)

Benchmarking your code and compare to the Distributions.jl function `logpdf`.

In [8]:
# benchmark the `logpdf` function in Distribution.jl
bm1 = @benchmark logpdf($mvn, $y)

BenchmarkTools.Trial: 
  memory estimate:  30.55 MiB
  allocs estimate:  5
  --------------
  minimum time:     17.297 ms (0.00% GC)
  median time:      19.323 ms (0.00% GC)
  mean time:        21.360 ms (11.06% GC)
  maximum time:     29.443 ms (22.08% GC)
  --------------
  samples:          234
  evals/sample:     1

In [10]:
using Profile
Profile.clear()
Profile.init()
@profile for i in 1:10000; logl!(obs, β, L, σ²); end
Profile.print(format=:flat)

 Count  Overhead File                    Line Function
 =====  ======== ====                    ==== ========
   212         0 In[10]                     4 macro expansion
   213         0 In[10]                     4 top-level scope
   178         0 In[2]                     14 logl!(::LmmObs{Float64}, ::Arra...
    10         0 In[2]                     19 logl!(::LmmObs{Float64}, ::Arra...
     3         0 In[2]                     20 logl!(::LmmObs{Float64}, ::Arra...
     3         0 In[2]                     28 logl!(::LmmObs{Float64}, ::Arra...
     2         0 In[2]                     36 logl!(::LmmObs{Float64}, ::Arra...
    16         0 In[2]                     44 logl!(::LmmObs{Float64}, ::Arra...
     1         0 @Base\array.jl           313 copyto!(::Array{Float64,1}, ::I...
     1         0 @Base\array.jl           318 copyto!(::Array{Float64,1}, ::I...
     3         1 @Base\array.jl           330 copyto!
     3         0 @Base\array.jl           361 similar
     1    

In [9]:
# benchmark your implementation
L = Matrix(cholesky(Σ).L)
bm2 = @benchmark logl!($obs, $β, $L, $σ²)

BenchmarkTools.Trial: 
  memory estimate:  224 bytes
  allocs estimate:  5
  --------------
  minimum time:     54.400 μs (0.00% GC)
  median time:      63.400 μs (0.00% GC)
  mean time:        66.537 μs (0.00% GC)
  maximum time:     530.000 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1

The points you will get is
$$
\frac{x}{1000} \times 30,
$$
where $x$ is the speedup of your program against the standard method.

In [10]:
# this is the points you'll get
clamp(median(bm1).time / median(bm2).time / 1000 * 30, 0, 30)

9.143257097791798

In [11]:
median(bm1).time / median(bm2).time

304.77523659305996

**Hint**: Apparently I am using 1000 as denominator because I expect your code to be at least $1000 \times$ faster than the standard method.

## Q5 Memory (30 pts)

You want to avoid memory allocation in the "hot" function `logl!`. You will lose 1 point for each `1 KiB = 1024 bytes` memory allocation. In other words, the points you get for this question is

In [11]:
clamp(30 - median(bm2).memory / 1024, 0, 30)

29.78125

**Hint**: I am able to reduce the memory allocation to 0 bytes.

## Q6 Misc (15 pts)

Coding style, Git workflow, etc. For reproducibity, make sure we (TA and myself) can run your Jupyter Notebook. That is how we grade Q4 and Q5. If we cannot run it, you will get zero points.